In [ ]:
import pymysql
import datetime

# Koneksi ke database
def get_connection():
    return pymysql.connect(
        host='afl2ht.h.filess.io',
        port=61002,
        user='sistemtkdb_usefulheor',
        password='382f83f60b7560c62cad795c7e8b88ca8f9e8626',
        db='sistemtkdb_usefulheor',
        cursorclass=pymysql.cursors.DictCursor
    )

# Konversi teks ke nilai angka
def label_ke_nilai(label):
    return {
        'BB': 1,
        'MB': 2,
        'BSH': 3,
        'BSB': 4
    }.get(label, 0)

# Konversi angka ke teks
def nilai_ke_label(nilai):
    return {
        1: 'BB',
        2: 'MB',
        3: 'BSH',
        4: 'BSB'
    }.get(nilai, 'Tidak Diketahui')

# Fungsi utama sistem pakar
def jalankan_sistem_pakar(id_siswa):
    conn = get_connection()
    cursor = conn.cursor()

    # Ambil data siswa
    cursor.execute("SELECT nama_lengkap FROM identitas_anak WHERE id = %s", (id_siswa,))
    siswa = cursor.fetchone()
    if not siswa:
        return f"Siswa dengan ID {id_siswa} tidak ditemukan."

    nama_siswa = siswa['nama_lengkap']

    # Ambil semua lingkup perkembangan
    cursor.execute("SELECT id, nama_lingkup FROM lingkup_perkembangan")
    lingkup_list = cursor.fetchall()

    hasil_lingkup = {}
    rekomendasi = []
    catatan = []

    tanggal_proses = datetime.datetime.now().strftime('%Y-%m-%d')

    for lingkup in lingkup_list:
        id_lingkup = lingkup['id']
        nama_lingkup = lingkup['nama_lingkup']

        # Ambil penilaian dari detail_asesmen berdasarkan lingkup
        cursor.execute("""
            SELECT da.skala_nilai
            FROM detail_asesmen da
            JOIN asesmen a ON da.id_asesmen = a.id_asesmen
            JOIN modul_ajar ma ON da.modulajar_id = ma.id
            WHERE a.id_siswa = %s AND ma.lingkup_id = %s
        """, (id_siswa, id_lingkup))
        nilai_list = cursor.fetchall()

        # Ubah nilai enum ke angka
        nilai_angka = [label_ke_nilai(n['skala_nilai']) for n in nilai_list if n['skala_nilai']]

        if not nilai_angka:
            hasil_lingkup[nama_lingkup] = 'Tidak ada data'
            continue

        # Hitung rata-rata
        total = sum(nilai_angka)
        avg = total / len(nilai_angka)
        avg_rounded = round(avg)

        hasil_lingkup[nama_lingkup] = nilai_ke_label(avg_rounded)

        # Buat rekomendasi bila nilai rendah
        if avg_rounded <= 2:
            rekomendasi.append(f"Perlu bimbingan tambahan di area {nama_lingkup}")
            catatan.append(f"- Perlu latihan tambahan pada aspek {nama_lingkup}")

    cursor.close()
    conn.close()

    return {
        'id_siswa': id_siswa,
        'nama_siswa': nama_siswa,
        'tanggal_proses': tanggal_proses,
        'hasil_per_lingkup': hasil_lingkup,
        'rekomendasi': rekomendasi,
        'catatan': catatan
    }

# Pemanggilan langsung (untuk testing di file .py)
if __name__ == "__main__":
    hasil = jalankan_sistem_pakar(1)  # Ganti dengan ID siswa yang valid

    print(f"\nID Siswa      : {hasil['id_siswa']}")
    print(f"Nama Siswa    : {hasil['nama_siswa']}")
    print(f"Tanggal Proses: {hasil['tanggal_proses']}")
    print("\n=== Hasil Penilaian per Lingkup Perkembangan ===")
    for lingkup, nilai in hasil['hasil_per_lingkup'].items():
        print(f"- {lingkup}: {nilai}")

    print("\n=== Rekomendasi Awal ===")
    for rek in hasil['rekomendasi']:
        print(f"- {rek}")

    print("\n=== Catatan Sistem Pakar ===")
    for note in hasil['catatan']:
        print(note)



ID Siswa      : 1
Nama Siswa    : Putri Halimah
Tanggal Proses: 2025-06-25

=== Hasil Penilaian per Lingkup Perkembangan ===
- Nilai Agama dan Moral: BSH
- Fisik-motorik Kasar: MB
- Fisik-motorik Halus: MB
- Fisik-motorik Kesehatan dan Perilaku Keselamatan: BSH
- Kognitif - Belajar dan Pemecahan Masalah: BSH
- Kognitif - Berfikir Logis: MB
- Kognitif - Berfikir Simbolik: BSH
- Bahasa - Memahami bahasa: BSH
- Bahasa - Mengungkapkan Bahasa: BSH
- Bahasa - Keaksaraan: MB

=== Rekomendasi Awal ===
- Perlu bimbingan tambahan di area Fisik-motorik Kasar
- Perlu bimbingan tambahan di area Fisik-motorik Halus
- Perlu bimbingan tambahan di area Kognitif - Berfikir Logis
- Perlu bimbingan tambahan di area Bahasa - Keaksaraan

=== Catatan Sistem Pakar ===
- Perlu latihan tambahan pada aspek Fisik-motorik Kasar
- Perlu latihan tambahan pada aspek Fisik-motorik Halus
- Perlu latihan tambahan pada aspek Kognitif - Berfikir Logis
- Perlu latihan tambahan pada aspek Bahasa - Keaksaraan


In [10]:
hasil = jalankan_sistem_pakar(1)  # Ganti dengan ID siswa yang valid

# Format tampilan hasil
print(f"\nID Siswa      : {hasil['id_siswa']}")
print(f"Nama Siswa    : {hasil['nama_siswa']}")
print(f"Tanggal Proses: {hasil['tanggal_proses']}")
print("\n=== Hasil Penilaian per Lingkup Perkembangan ===")
for lingkup, nilai in hasil['hasil_per_lingkup'].items():
    print(f"- {lingkup}: {nilai}")

print("\n=== Rekomendasi Awal ===")
for rek in hasil['rekomendasi']:
    print(f"- {rek}")

print("\n=== Catatan Sistem Pakar ===")
for note in hasil['catatan']:
    print(note)



ID Siswa      : 1
Nama Siswa    : Putri Halimah
Tanggal Proses: 2025-06-25

=== Hasil Penilaian per Lingkup Perkembangan ===
- Nilai Agama dan Moral: BSH
- Fisik-motorik Kasar: MB
- Fisik-motorik Halus: MB
- Fisik-motorik Kesehatan dan Perilaku Keselamatan: BSH
- Kognitif - Belajar dan Pemecahan Masalah: BSH
- Kognitif - Berfikir Logis: MB
- Kognitif - Berfikir Simbolik: BSH
- Bahasa - Memahami bahasa: BSH
- Bahasa - Mengungkapkan Bahasa: BSH
- Bahasa - Keaksaraan: MB

=== Rekomendasi Awal ===
- Perlu bimbingan tambahan di area Fisik-motorik Kasar
- Perlu bimbingan tambahan di area Fisik-motorik Halus
- Perlu bimbingan tambahan di area Kognitif - Berfikir Logis
- Perlu bimbingan tambahan di area Bahasa - Keaksaraan

=== Catatan Sistem Pakar ===
- Perlu latihan tambahan pada aspek Fisik-motorik Kasar
- Perlu latihan tambahan pada aspek Fisik-motorik Halus
- Perlu latihan tambahan pada aspek Kognitif - Berfikir Logis
- Perlu latihan tambahan pada aspek Bahasa - Keaksaraan
